In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
database_path = Path('/home/s175668/raid/Praca-Magisterska/dataset/pose3d/under/')
json_files = database_path.glob('*.json')

In [ ]:
files = []
for file in json_files:
    file_name = str(file).rsplit('/', 1)[-1]
    camera = str(file).rsplit('_', 2)[-2]
    label = int(file_name[-6])
    shoter = str(file).split('_', 2)[1]
    if int(file_name.split('_')[0]) in [574, 682, 1307, 1341]:
        continue
    files.append([file_name, shoter, camera, label])
    

In [4]:
df = pd.DataFrame(files, columns=['name', 'shoter', 'camera', 'label'])

In [13]:
df

,name,shoter,camera,label
0,0887_adam_under_0.json,adam,under,0
1,1453_adam_under_1.json,adam,under,1
2,0729_adam_under_0.json,adam,under,0
3,1500_kuba_under_0.json,kuba,under,0
4,0689_adam_under_0.json,adam,under,0
...,...,...,...,...
1918,1061_adam_under_1.json,adam,under,1
1919,0961_adam_under_1.json,adam,under,1
1920,0058_krystian_skill_under_1.json,krystian,under,1
1921,1047_adam_under_0.json,adam,under,0


In [ ]:
len(df.loc[df['label'] == 0])

In [ ]:
krystian_0 = df.loc[(df['shooter'] == 'krystian') & (df['label'] == 0)]

In [ ]:
krystian_1 = ase.loc[(database['shooter'] == 'krystian') & (database['label'] == 1) &(database['camera'] == 'under')]

In [ ]:
krystian_0['path'].iloc[:int(len(krystian_0)*0.2)].values

In [ ]:
krystian_0['path'].iloc[:int(len(krystian_0)*0.2)].values

## SPLIT ONLY TO TEST AND VAL - MANIPULATE LEN

In [5]:
train = []
val = []
# for s in shooter:
shooter = ['adam', 'krystian', 'kuba', 'gustaw']
for s in shooter:
    for i in range(2):
        shooter_df = df.loc[(df['shoter'] == s) & (df['label'] == i) & (df['camera']=='under')]
        if s == 'adam' and i == 1:
            print('lala')
            shooter_df = shooter_df.iloc[200:]
        train.extend(shooter_df.iloc[int(len(shooter_df)*0.15):].values)
        val.extend(shooter_df.iloc[:int(len(shooter_df)*0.15)].values)

# import csv
# header = ['path']
# with open(str(database_path.parent / 'fold_under_train.csv'), 'w') as f:
#     write = csv.writer(f)
#     write.writerow(header)
#     write.writerows(train)
    
# with open(str(database_path.parent / 'fold_under_val.csv'), 'w') as f:
#     write = csv.writer(f)
#     write.writerow(header)
#     write.writerows(val)

lala


In [6]:
len(train)

1466

In [7]:
len(val)

256

In [8]:
import random
random.shuffle(train)
random.shuffle(val)

In [9]:
train_df = pd.DataFrame(train, columns = ['name', 'shoter', 'camera', 'label'])
val_df = pd.DataFrame(val, columns = ['name', 'shoter', 'camera', 'label'])

In [12]:
train_df.to_csv('/home/s175668/raid/Praca-Magisterska/dataset/pose3d/train_under.csv', index=False)
val_df.to_csv('/home/s175668/raid/Praca-Magisterska/dataset/pose3d/val_under.csv', index=False)

# ADD TEST

In [ ]:
# test for one shooter
test = []
for s in ['gustaw']:
    for i in range(2):
        df = database.loc[(database['shooter'] == s) & (database['label'] == i) &(database['camera'] == 'under')]
        test.extend(df['path'].iloc[:int(len(df)*0.2)].values)
        
test = [[t] for t in test]
with open(str(database_path.parent / 'fold_4_test.csv'), 'w') as f:
    write = csv.writer(f)
    write.writerow(header)
    write.writerows(test)

In [ ]:
csv = pd.read_csv(database_path.parent / 'fold_1_train.csv')
csv

/home/s175668/raid/Praca-Magisterska/csv_extracted/conf_th_60/766_krystian_under_1.csv change it to label 1

# INTERPOLATION CHECK

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import math


from torch.utils.data import Dataset
from pathlib import Path

In [ ]:
df = pd.read_csv('/home/s175668/raid/Praca-Magisterska/csv_extracted/th_60_kp_with_conf/168_tomek_under_1.csv')
# pd.set_option('display.max_columns', None)

df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
target = df.iloc[0,-1]
conf = df.iloc[:, 4:-1:3].values
df = df.drop(df.columns[[i for i in range(4, 39, 3)]], axis = 1)
df = df.drop(['TARGET'], axis=1)
cords = df.iloc[:, :].values

In [ ]:
cords.shape

In [ ]:
def interpolation(cords):
    index = np.where(cords == 0)
    for c in range(cords.shape[1]):
        cords_to_interp = cords[:, c]

        timestep = np.where(cords_to_interp > 0)[0]
        index_to_interp = np.where(cords_to_interp == 0)[0]
        cords_to_interp = np.delete(cords_to_interp, index_to_interp)
        
        interpolated = np.interp(index_to_interp, timestep, cords_to_interp)
        
        for i, index in enumerate(index_to_interp):
            cords[index, c] = interpolated[i]
        
    return cords
        

In [ ]:
def calulate_vector_length(cords_x, cords_y, x0, y0):
    vector_lenth = np.zeros((14,12))
    for t in range(14):
        for kp in range(12):
            x1, y1 = cords_x[t, kp], cords_y[t,kp]
            vector_lenth[t, kp] = math.sqrt((x1-x0)**2+(y1-y0)**2)
    
    return vector_lenth

In [ ]:
conf = interpolation(conf)
cords = interpolation(cords)
print(cords.shape)
nose_x, nose_y = cords[0, 0], cords[0, 1]
cords_x, cords_y = cords[:, 2::2], cords[:, 3::2]

lh_x, rh_x = cords_x[0,6], cords_x[0,7]
lh_y, rh_y = cords_y[0,6], cords_y[0,7]
x0, y0 = abs(lh_x - rh_x)/2, abs(lh_y - rh_y)/2

normalize_vector = math.sqrt((nose_x-x0)**2+(nose_y-y0)**2)
print(normalize_vector)

vector_length = calulate_vector_length(cords_x, cords_y, x0, y0) / normalize_vector

In [ ]:
import matplotlib.pyplot as plt
  
# naming the x axis
plt.plot(cords_x[5], cords_y[5], 'o', color='black');

In [ ]:
import matplotlib.pyplot as plt
  
# naming the x axis
plt.plot(x, y*-1, 'o', color='black');

In [ ]:
x =cords_x[6] - x0

In [ ]:
y = cords_y[6] - y0

## CREATE DATABASE 3D

In [ ]:
from pathlib import Path

files_path = Path('/home/s175668/raid/Praca-Magisterska/csv_extracted/3d_mmpose_14/')

In [ ]:
files = files_path.glob('*')

In [ ]:
file_info = []
i = 0 
for file in files:
    file_name = str(file).rsplit('/', 1)[-1]
    
    if 'niecelny' in file_name or file_name[-6] == '0':
        made = 0 
    else:
        made = 1
    
    if 'side' in file_name or 'site' in file_name:
        camera = 'side'
    else:
        camera = 'under'
    
    if 'krystian' in file_name:
        shooter = 2
    elif 'kuba' in file_name:
        shooter = 1
    else:
        shooter = 0
    
    file_info.append([file_name, camera, made, shooter])
    
df = pd.DataFrame(file_info, columns = ['file', 'camera', 'made', 'skill'])

In [ ]:
df_dropped = df.drop(['skill'], axis=1)

In [ ]:
len(df_dropped)*0.15

In [ ]:
df_val = df_dropped[:int(len(df_dropped)*0.15)]

In [ ]:
df_train = df_dropped[int(len(df_dropped)*0.15):]

In [ ]:
df_train.to_csv('/home/s175668/raid/Praca-Magisterska/csv_extracted/3d_mmpose_14/fold_train.csv', index = False)
df_val.to_csv('/home/s175668/raid/Praca-Magisterska/csv_extracted/3d_mmpose_14/fold_val.csv', index = False)
df.to_csv('/home/s175668/raid/Praca-Magisterska/csv_extracted/3d_mmpose_14/database.csv', index = False)

In [ ]:
df_val.reset_index(drop=True)

# DATABASE LAST VERSION


In [42]:
broken_files = [574, 682, 1307, 1341, 4419 , 3661, 2224, 2762, 3253, 3248, 3360, 3678, 2254, 2950]

In [43]:
database = pd.read_csv('/home/s175668/raid/Praca-Magisterska/dataset/database.csv')

In [44]:
one_perspective = database[~(database['camera'] == 'side')].drop(columns='camera')

In [45]:
made = len(one_perspective[one_perspective['made'] == 1])
not_made = len(one_perspective) - made

clean = len(one_perspective[one_perspective['clean'] == 1])
not_clean = len(one_perspective) - clean

In [46]:
files = []
for _, row in database.iterrows():
    if int(row.file.split('_', 1)[0]) not in broken_files:
        files.append(row.values)

In [47]:
database_filtered = pd.DataFrame(files, columns=database.columns)

In [48]:
database_filtered

,file,shoter,camera,made,clean,skill
0,0001_krystian_under_1,krystian,under,1,1,2
1,0001_krystian_side_1,krystian,side,1,1,2
2,0002_krystian_under_1,krystian,under,1,1,2
3,0002_krystian_side_1,krystian,side,1,1,2
4,0003_krystian_under_1,krystian,under,1,1,2
...,...,...,...,...,...,...
9017,4523_kuba_side_1,kuba,side,1,1,1
9018,4524_gustaw_under_0,gustaw,under,0,0,0
9019,4524_gustaw_side_0,gustaw,side,0,0,0
9020,4525_kuba_under_0,kuba,under,0,0,1


In [49]:
database

,file,shoter,camera,made,clean,skill
0,0001_krystian_under_1,krystian,under,1,1,2
1,0001_krystian_side_1,krystian,side,1,1,2
2,0002_krystian_under_1,krystian,under,1,1,2
3,0002_krystian_side_1,krystian,side,1,1,2
4,0003_krystian_under_1,krystian,under,1,1,2
...,...,...,...,...,...,...
9045,4523_kuba_side_1,kuba,side,1,1,1
9046,4524_gustaw_under_0,gustaw,under,0,0,0
9047,4524_gustaw_side_0,gustaw,side,0,0,0
9048,4525_kuba_under_0,kuba,under,0,0,1


In [50]:
len(database_filtered[(database_filtered['shoter'] == 'kuba')])*0.2

580.4

In [51]:
len(database_filtered[(database_filtered['shoter'] == 'gustaw')])*0.2

457.20000000000005

In [52]:
len(database_filtered[database_filtered['shoter'] == 'adam'])*0.2

410.40000000000003

In [53]:
len(database_filtered.loc[(database_filtered['shoter'] == 'krystian')])*0.2

356.40000000000003

In [54]:
len(database_filtered.loc[(database_filtered['shoter'] == 'krystian')&(database_filtered['made'] == 0)])*0.2

125.60000000000001

In [55]:
train = []
val = []
shooters = ['adam', 'krystian', 'kuba', 'gustaw']
for s in shooters:
    print(s)
    for m in range(2):
        print(m)
        df = database_filtered.loc[(database_filtered['shoter'] == s)&(database_filtered['made'] == m)]
        df = df.drop(columns=['clean', 'skill'])
        train.extend(df.iloc[int(len(df)*0.2):].values)
        val.extend(df.iloc[:int(len(df)*0.2)].values)

adam
0
1
krystian
0
1
kuba
0
1
gustaw
0
1


In [56]:
train_df = pd.DataFrame(train, columns = ['name', 'shoter', 'camera', 'label'])
val_df = pd.DataFrame(val, columns = ['name', 'shoter', 'camera', 'label'])

In [57]:
train_df

,name,shoter,camera,label
0,0655_adam_side_0,adam,side,0
1,0656_adam_under_0,adam,under,0
2,0656_adam_side_0,adam,side,0
3,0667_adam_under_0,adam,under,0
4,0667_adam_side_0,adam,side,0
...,...,...,...,...
7217,4498_gustaw_side_1,gustaw,side,1
7218,4510_gustaw_under_1,gustaw,under,1
7219,4510_gustaw_side_1,gustaw,side,1
7220,4519_gustaw_under_1,gustaw,under,1


In [58]:
val_df

,name,shoter,camera,label
0,0437_adam_under_0,adam,under,0
1,0437_adam_side_0,adam,side,0
2,0440_adam_under_0,adam,under,0
3,0440_adam_side_0,adam,side,0
4,0442_adam_under_0,adam,under,0
...,...,...,...,...
1795,2392_gustaw_under_1,gustaw,under,1
1796,2392_gustaw_side_1,gustaw,side,1
1797,2393_gustaw_under_1,gustaw,under,1
1798,2393_gustaw_side_1,gustaw,side,1


In [59]:
train_df.to_csv('/home/s175668/raid/Praca-Magisterska/dataset/folds/made/train_made_balance.csv', index=False)

In [60]:
val_df.to_csv('/home/s175668/raid/Praca-Magisterska/dataset/folds/made/val_made_balance.csv', index=False)

In [61]:
len(train_df[train_df['label'] == 0])/ len(train_df)

0.4706452506230961